In [579]:
import numpy as np
import importlib
import Layout
import Greedy

import copy 

#Reset libreria
importlib.reload(px)
importlib.reload(Greedy)
importlib.reload(Layout)

def showtime2(path):
    layout=px.read(path)
    
    
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    Greedy.lazy_greedy(layout)
    seleccionada = SSRC(layout)
 
    while seleccionada != -1:
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        if verbose: print("selected:",layout.stacks[seleccionada])

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)

        #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
        SFpp(layout,seleccionada,rank, layout.H-len(layout.stacks[seleccionada]) )#nb_unsorted_elements)

        Greedy.lazy_greedy(layout)
        #Se vuelve a seleccionar una columna
        seleccionada = SSRC(layout)

    #print("SELECCIONADA:", seleccionada)
    #print(len(layout.moves),"\t",layout.unsorted_stacks)
    if verbose: 
        for s in layout.stacks: print(s)
    return len(layout.moves)
    #print()

In [580]:
#selector de columnas a desmantelar
def SSRC(layout):
    best_ev = 0; s_o=-1
    for i in range(len(layout.stacks)):
        if layout.is_sorted_stack(i): continue
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        ev = 10000 - 100*len(layout.stacks[i]) - prom

        if ev > best_ev:
            best_ev = ev
            s_o = i
    return s_o

In [581]:
def SDpp(layout, s_o, rank, n_elements=100):
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = np.maximum(len(ss_o)-n_elements,0)
    
    while len(layout.stacks[s_o]) > fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack, rank=rank)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            if verbose: 
                for s in layout.stacks: print(s)
                print(s_d,(pos - slack) - rank[top],s_o, rank)
            Greedy.fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

In [582]:
import sys
def SFpp(layout, s_d, rank, n):
    ub = Layout.gvalue(layout.stacks[s_d])
    cont = 0
    for r in rank:
        while True:
            (s, pos) = Greedy.search_highest(layout, r, ub, s_d)  
            #if verbose:
                #print("r,ub:",r,ub)
                #print(layout.stacks[s][pos])
                
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            ub=layout.stacks[s][pos]-1
                      
            c=Greedy.force_move(layout, s, pos, s_d)
            
            
            if c == None:    
                if ub<r: 
                    for stack in layout.stacks:
                        print(stack)
                    print(str(ub+1)+ " del stack " +str(s)+ " no se puede volver a colocar en "+str(s_d)+"!!");
                    sys.exit()
            else: 
                cont+=1
                if cont == n: return
                if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

In [583]:
import Greedy as gr
import importlib
importlib.reload(Greedy)
importlib.reload(Layout)
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("Instancias") if not isfile(join("Instancias", f))]

verbose = False
tot_moves=0
for f in onlyfiles:
    if f == 'BF' or f== '3-x' : continue
    #print(f)
    nmoves=0
    for i in range(1,41):
        filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
        nmoves += showtime2(filename)
    tot_moves += nmoves
    print (f,nmoves/ 40.0);
print("prom:",tot_moves/(21.0*40.0))

10-10 160.975
10-6 124.775
3-3 12.9
3-4 13.75
3-5 13.925
3-6 14.9
3-7 16.325
3-8 16.925
4-4 23.475
4-5 25.65
4-6 26.775
4-7 30.15
5-10 54.55
5-4 35.05
5-5 37.075
5-6 41.45
5-7 42.25
5-8 48.8
5-9 51.9
6-10 73.65
6-6 54.425
prom: 43.79404761904762


![image](https://i.imgur.com/XsBW9YR.png)

In [584]:
verbose = True
f='3-3'
for i in range(1,41):
    filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
    #print(i,showtime2(filename))
i=14
filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
print(i,showtime2(filename))

[6, 7, 4]
[1, 3, 5]
[2, 9, 8]


selected: [1, 3, 5]
[6, 7, 4, 5]
[]
[2, 9, 8, 3, 1]


[6, 7]
[5, 4, 3, 1]
[2, 9, 8]


selected: [6, 7]
[]
[5, 4, 3, 1, 7]
[2, 9, 8, 6]


[7, 6]
[5, 4, 3, 1]
[2, 9, 8]


selected: [2, 9, 8]
[7, 6, 8, 9]
[5, 4, 3, 1, 2]
[]


[7, 6]
[5, 4, 3, 1]
[9, 8, 2]
14 18
